In [20]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.chat_models.openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain

In [15]:
import os

In [16]:
os.environ["OPENAI_API_KEY"] = ""

In [17]:
# Load dos modelos (Embeddings e LLMs)

embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(model_name="gpt-3.5-turbo", max_tokens=200)

/var/folders/t9/2wfh8wr96y910pmrrcxplfb00000gn/T/ipykernel_21908/1097145635.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", max_tokens=200)


In [19]:
# Carregando o PDF

pdf_link = "book.pdf"

loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()

In [23]:
#Separar em Chunks

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 4000,
    chunk_overlap = 20,
    length_function = len,
    add_start_index = True
)

chunks = text_splitter.split_documents(pages)

In [29]:
#Salvar no Vector DB - Chroma

db = Chroma.from_documents(chunks, embedding=embeddings_model, persist_directory="text_index2")

In [33]:
# Carregar DB

vectordb = Chroma(persist_directory="text_index2", embedding_function=embeddings_model)

# Load Retriever

retriever = vectordb.as_retriever(search_kwargs={"k": 3})

# Construção da cadeira de prompt para chamada do LLM

chain = load_qa_chain(llm, chain_type="stuff")

/var/folders/t9/2wfh8wr96y910pmrrcxplfb00000gn/T/ipykernel_21908/4090480388.py:11: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(llm, chain_type="stuff")


In [39]:
def ask(question):
    context = retriever.get_relevant_documents(question)
    answer = (chain({"input_documents": context, "question": question}, return_only_outputs=True))['output_text']
    return answer, context

In [48]:
user_question = input("User: ")
answer, context = ask(user_question)
print("Answer: ", answer)

Answer:  I'm sorry, but I can't see the content that was scanned with CamScanner. How can I assist you today?


In [47]:
print(context[0])

page_content='Scanned with CamScanner' metadata={'page': 448, 'page_label': '449', 'source': 'book.pdf', 'start_index': 0}
